In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import re

In [2]:
#Loading raw data
N = pd.read_excel('neighbourhood-profiles-2021-158-model.xlsx').T

In [3]:
#Rearranging
N.columns = N.iloc[0]
N = N.drop('Neighbourhood Name')
N.columns = N.columns.str.strip()

In [4]:
#Creating a new dataframe
Neighbourhoods = pd.DataFrame()

In [5]:
#Neighbourhood number column
Neighbourhoods['Neighbourhood Number'] = N['Neighbourhood Number']
#Neighbourhood Name
Neighbourhoods['Neighbourhood Name'] = N.index

In [6]:
#Calculating total population based on age groups
Aged_0_to_14 = N.loc[:, '0 to 14 years'].iloc[:, 0]
Aged_15_to_64 = N.loc[:, '15 to 64 years'].iloc[:, 0]
Aged_65_and_over = N.loc[:, '65 years and over'].iloc[:, 0]
Neighbourhoods['Population'] = Aged_0_to_14 + Aged_15_to_64 + Aged_65_and_over

In [7]:
#Other features
#Age
Neighbourhoods['Average Age'] = N['Average age of the population']
Neighbourhoods['Median Age'] = N['Median age of the population']
#Income
Neighbourhoods['Average Total Income in 2020'] = N['Average total income in 2020 among recipients ($)']
#Immigrants/Non-immigrants
Neighbourhoods['Immigrants'] = N['Immigrants']
Neighbourhoods['Immigrants_Percentage'] = Neighbourhoods['Immigrants'] / Neighbourhoods['Population']
Neighbourhoods['Non-Immigrants'] = N['Non-immigrants']
Neighbourhoods['Non-Immigrants_Percentage'] = Neighbourhoods['Non-Immigrants'] / Neighbourhoods['Population']
#Employment
Neighbourhoods['Employed'] = N['Employed']
Neighbourhoods['Employed_Percentage'] = Neighbourhoods['Employed'] / Neighbourhoods['Population']
Neighbourhoods['Unemployed'] = N['Unemployed']
Neighbourhoods['Unemployed_Percentage'] = Neighbourhoods['Unemployed'] / Neighbourhoods['Population']
#Education level
Neighbourhoods['No Degree'] = N['No certificate, diploma or degree'].iloc[:,0]
Neighbourhoods['No Degree_Percentage'] = Neighbourhoods['No Degree'] / Neighbourhoods['Population']
Neighbourhoods['High School Diploma'] = N['High (secondary) school diploma or equivalency certificate'].iloc[:,0]
Neighbourhoods['High School Diploma_Percentage'] = Neighbourhoods['High School Diploma'] / Neighbourhoods['Population']
Neighbourhoods['Postsecondary Certificate'] = N['Postsecondary certificate, diploma or degree'].iloc[:,0]
Neighbourhoods['Postsecondary Certificate_Percentage'] = Neighbourhoods['Postsecondary Certificate'] / Neighbourhoods['Population']
Neighbourhoods['Bachelor\'s'] = N['Bachelor\'s degree'].iloc[:,0]
Neighbourhoods['Bachelor\'s_Percentage'] = Neighbourhoods['Bachelor\'s'] / Neighbourhoods['Population']
Neighbourhoods['Master\'s'] = N['Master\'s degree'].iloc[:,0]
Neighbourhoods['Master\'s_Percentage'] = Neighbourhoods['Master\'s'] / Neighbourhoods['Population']
Neighbourhoods['Doctorate'] = N['Earned doctorate'].iloc[:,0]
Neighbourhoods['Doctorate_Percentage'] = Neighbourhoods['Doctorate'] / Neighbourhoods['Population']

In [8]:
#Loading theft and geomertry data for merging

In [9]:
Neighbourhood_geometry = gpd.read_file('Neighbourhoods.geojson')
geoBikeAndNeighbourhood = gpd.read_file('bike_with_n_and_p.json')
geoCarAndNeighbourhood = gpd.read_file('car_with_n_and_p.json')

In [10]:
geoBikeAndNeighbourhood = geoBikeAndNeighbourhood.rename(columns={'neighborhood_name': 'neighbourhood_name'})
geoCarAndNeighbourhood = geoCarAndNeighbourhood.rename(columns={'neighborhood_name': 'neighbourhood_name'})
Neighbourhood_geometry = Neighbourhood_geometry.rename(columns={'AREA_NAME': 'neighbourhood_name'})

In [11]:
# Function to clean the neighborhood name: lowercased and only letters
def clean_neighborhood_name(name):
    # Convert to lowercase and remove non-alphabetic characters
    return re.sub(r'[^a-zA-Z]', '', name.lower())

bike_thefts_agg = geoBikeAndNeighbourhood.groupby('neighbourhood_name').agg(
    bike_thefts = ('neighbourhood_name', 'size'),
    neighbourhood_area = ('neighborhood_area', 'first'),
    park_area = ('park_area', 'first'),
    park_density = ('park_density', 'first'),
    park_count = ('park_count', 'first'),# Assuming 'neighbourhood_area' is the same for all crimes in the same neighborhood
).reset_index()
car_thefts_agg = geoCarAndNeighbourhood.groupby('neighbourhood_name').size().reset_index(name='car_thefts')

In [12]:
# Merging neighbourhood and crime data (eliminating name misspelling)
Neighbourhoods['clean_name'] = Neighbourhoods['Neighbourhood Name'].apply(clean_neighborhood_name)
bike_thefts_agg['clean_name'] = bike_thefts_agg['neighbourhood_name'].apply(clean_neighborhood_name)
car_thefts_agg['clean_name'] = car_thefts_agg['neighbourhood_name'].apply(clean_neighborhood_name)
Neighbourhood_geometry['clean_name'] = Neighbourhood_geometry['neighbourhood_name'].apply(clean_neighborhood_name)
Neighbourhood_geometry = Neighbourhood_geometry[['clean_name', 'geometry']]

#Merge datasets using the cleaned names
NeighbourhoodsWithCrime = pd.merge(Neighbourhoods, bike_thefts_agg, 
                                   on='clean_name', how='left')

NeighbourhoodsWithCrime = pd.merge(NeighbourhoodsWithCrime, car_thefts_agg, 
                                   on='clean_name', how='left')

NeighbourhoodsWithCrime = pd.merge(NeighbourhoodsWithCrime, Neighbourhood_geometry, 
                                   on='clean_name', how='left')

#Drop the 'clean_name' column and keep the original 'Neighbourhood Name'
NeighbourhoodsWithCrime.drop('clean_name', axis=1, inplace=True)
NeighbourhoodsWithCrime.drop('neighbourhood_name_x', axis=1, inplace=True)
NeighbourhoodsWithCrime.drop('neighbourhood_name_y', axis=1, inplace=True)

In [13]:
NeighbourhoodsWithCrime = NeighbourhoodsWithCrime.rename(columns={'bike_thefts': 'Bike Thefts'})
NeighbourhoodsWithCrime = NeighbourhoodsWithCrime.rename(columns={'car_thefts': 'Car Thefts'})
NeighbourhoodsWithCrime = NeighbourhoodsWithCrime.rename(columns={'park_density': 'Park Density'})
NeighbourhoodsWithCrime = NeighbourhoodsWithCrime.rename(columns={'park_count': 'Park Count'})
NeighbourhoodsWithCrime = NeighbourhoodsWithCrime.rename(columns={'park_area': 'Park Area'})

In [14]:
NeighbourhoodsWithCrime.head()

,Neighbourhood Number,Neighbourhood Name,Population,Average Age,Median Age,Average Total Income in 2020,Immigrants,Immigrants_Percentage,Non-Immigrants,Non-Immigrants_Percentage,...,Master's_Percentage,Doctorate,Doctorate_Percentage,Bike Thefts,neighbourhood_area,Park Area,Park Density,Park Count,Car Thefts,geometry
0,1,West Humber-Clairville,33295,40.5,38,40560,18805,0.5648,11805,0.354558,...,0.04385,130,0.003904,55,3.161667e+07,2.328015e+06,7.363251,37.0,2696,"MULTIPOLYGON (((-79.59037 43.73401, -79.58942 ..."
1,2,Mount Olive-Silverstone-Jamestown,31350,37.8,36,35000,19810,0.631898,9620,0.306858,...,0.040351,90,0.002871,30,4.862431e+06,1.051879e+06,21.632779,22.0,581,"MULTIPOLYGON (((-79.57938 43.76222, -79.57925 ..."
2,3,Thistletown-Beaumond Heights,9850,41,39.2,41520,5210,0.528934,4055,0.411675,...,0.04467,15,0.001523,20,3.501726e+06,1.004922e+06,28.697908,10.0,237,"MULTIPOLYGON (((-79.56226 43.74373, -79.56234 ..."
3,4,Rexdale-Kipling,10375,41.9,42,42040,4820,0.464578,5080,0.489639,...,0.02747,30,0.002892,14,2.627713e+06,2.561669e+05,9.748665,6.0,187,"MULTIPOLYGON (((-79.56419 43.71229, -79.5667 4..."
4,5,Elms-Old Rexdale,9355,39.6,38.4,41240,4600,0.491716,4515,0.48263,...,0.031534,25,0.002672,16,3.073903e+06,7.735079e+05,25.163700,6.0,165,"MULTIPOLYGON (((-79.54677 43.73156, -79.54632 ..."


In [15]:
NeighbourhoodsWithCrime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   Neighbourhood Number                  158 non-null    object  
 1   Neighbourhood Name                    158 non-null    object  
 2   Population                            158 non-null    object  
 3   Average Age                           158 non-null    object  
 4   Median Age                            158 non-null    object  
 5   Average Total Income in 2020          158 non-null    object  
 6   Immigrants                            158 non-null    object  
 7   Immigrants_Percentage                 158 non-null    object  
 8   Non-Immigrants                        158 non-null    object  
 9   Non-Immigrants_Percentage             158 non-null    object  
 10  Employed                              158 non-null    object  
 11  Employ

In [18]:
NeighbourhoodsWithCrime = gpd.GeoDataFrame(NeighbourhoodsWithCrime, geometry='geometry')
NeighbourhoodsWithCrime.to_file('NeighbourhoodsWithCrime.json', driver="GeoJSON")

In [33]:
!git init
!git remote add origin <https://github.com/tamarm/Amateurs-in-the-Dataverse.git>

Reinitialized existing Git repository in C:/Users/tomer/Amateurs-in-the-Dataverse/.git/


The syntax of the command is incorrect.


In [34]:
!git add $(Neighbourhood_data_cleaning_and_merging.ipynb $0)

fatal: pathspec '$(Neighbourhood_data_cleaning_and_merging.ipynb' did not match any files


In [35]:
# Check the status
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [36]:
# Commit the file
!git commit -m "Add $(basename $0)"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [37]:
# Push the changes to the repository
!git push origin main

Everything up-to-date
